# Deadlock

In this demo we will work with two transactions.

1. We'll prepare Performance monitor for the deadlock monitoring (SQL Server:Locks - Number of Deadlocks/sec)
2. We'll execute two transactions
3. We'll show how to catch and handle the deadlock victim error

## Transactions in conflict
Following code looks twice the same, but it's not. Note, that the order of objects in switched.

Let's copy transactions to SSMS and let's execute it "slowly".

In [ ]:
-- following code must be divided between two SSMS query windows

-- first transaction
use Demo
go

begin tran
update dbo.SalesOrders set Subtotal = 1000 where id between 300000 and 310000
update dbo.SalesOrdersCopy set Subtotal = 1000 where id between 300000 and 310000

rollback

-- second transaction
use Demo
go

begin tran
update dbo.SalesOrdersCopy set Subtotal = 1000 where id between 300000 and 310000
update dbo.SalesOrders set Subtotal = 1000 where id between 300000 and 310000

rollback

## Handling the error number 1205

In [ ]:
use Demo
go

declare @attemptNo int = 3

while @attemptNo > 0
 begin
	begin try
		begin tran
		update dbo.SalesOrdersCopy set Subtotal = 1000 where id between 300000 and 310000
		update dbo.SalesOrders set Subtotal = 1000 where id between 300000 and 310000
		commit
	end try
	begin catch
		if ERROR_NUMBER() = 1205
		 begin
			set @attemptNo -= 1
		 end
		else
		 begin
			set @attemptNo = 0
			if @@TRANCOUNT > 0
				rollback;
			throw;
		 end
	end catch
 end